In [ ]:
"""This module implements the Fréchet regression method from Petersen and Müller 2019.

The method is implemented in the context of estimation in the space of distribution
functions. Also, a simulation study is designed to compare with the methods outlined in
Petersen and Müller 2016.

"""

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import quadprog
from scipy.stats import norm

from frechet_fda.function_class import Function
from frechet_fda.tools.function_tools import mean_func

In [ ]:
N = 200
seed = 28071995
predictor = np.random.default_rng(seed).uniform(-1, 1, N)
predictor.sort()

In [ ]:
def mu_from_x(x, mu0, beta, v1):
    """Sample mus from the first simulation setting in Petersen & Müller (2019)."""
    return np.random.default_rng(seed).normal(loc=mu0 + beta * x, scale=v1)

In [ ]:
mus = mu_from_x(predictor, 0, 3, 0.25)

In [ ]:
def sigma_from_x(x, sigma0, gamma, v2):
    """Sample sigmas from the first simulation setting Petersen & Müller (2019)."""
    sh = (sigma0 + gamma * x) ** 2 / v2
    sc = v2 / (sigma0 + gamma * x)

    return np.random.default_rng(seed).gamma(shape=sh, scale=sc)

In [ ]:
sigmas = sigma_from_x(predictor, 3, 0.5, 2)

In [ ]:
grid_size = 500
delta = 0
u = np.linspace(delta, 1 - delta, grid_size)

In [ ]:
def gen_y_qf(mu, sigma, eval_points):
    """Generate quantile function of Y_i given X_i."""
    ys = (
        mu[..., np.newaxis]
        + sigma[..., np.newaxis] * norm.ppf(eval_points)[np.newaxis, ...]
    )
    return [Function(eval_points, y) for y in ys]

In [ ]:
qfs = gen_y_qf(mus, sigmas, u)

In [ ]:
qfs[0].y.shape

In [ ]:
qfs[0].compare(qfs[199], label_self="x = -1", label_other="x = 1")

In [ ]:
fig, ax = plt.subplots()
ax.plot(
    qfs[0].x,
    qfs[0].y,
    label=f"Mu = {round(mus[0], 2)}, Sigma = {round(sigmas[0], 2)}",
)
ax.plot(
    qfs[199].x,
    qfs[199].y,
    label=f"Mu = {round(mus[199], 2)}, Sigma = {round(sigmas[199], 2)}",
)
plt.grid()
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
for qf in qfs[:6]:
    pdf = qf.invert().differentiate()
    ax.plot(
        pdf.x,
        pdf.y,
    )
plt.grid()
plt.show()

In [ ]:
def _empirical_weight_function(x_eval, predictor_values):
    """Weight function to compute weighted Fréchet mean."""
    predictor_values = np.atleast_2d(predictor_values)
    means = np.mean(predictor_values, axis=-1)
    cov_matrix = (
        predictor_values @ predictor_values.transpose() / predictor_values.shape[-1]
    )
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    return 1 + (
        (predictor_values - means[..., np.newaxis]).transpose()
        @ inv_cov_matrix
        @ (x_eval - means)
    )

In [ ]:
def qf_tilde(quantile_funcs, x, predictor_values):
    """Estimator for quantile function."""
    return mean_func(_empirical_weight_function(x, predictor_values) * quantile_funcs).y

In [ ]:
# Take only very tenth value of predictor grid to calculate estimate
x_grid = predictor[::10]
x_grid.shape

In [ ]:
def solve_qp(u, x_grid, x_observed, quantile_functions) -> list[Function]:
    """Sets up quadratic programming problem.

    quantile_functions need to have a support of len(u).
    """
    grid_size = len(u)
    qp_g = np.identity(grid_size)  # make sure P is symmetric
    qp_c = np.eye(grid_size, grid_size - 1, k=-1) - np.eye(grid_size, grid_size - 1)
    qp_b = np.zeros(grid_size - 1)
    estimates = []
    for x in x_grid:
        qp_a = qf_tilde(quantile_functions, x, x_observed)
        constraints_check = qp_a[1:] - qp_a[:-1]
        # remove nans
        constraints_check = constraints_check[~np.isnan(constraints_check)]
        if np.all(constraints_check > 0):
            # If estimator valid qf, it is the optimal solution
            solution = qp_a
        else:
            # Else, find closest vector to estimator that is valid solution
            solution = quadprog.solve_qp(qp_g, qp_a, qp_c, qp_b)[0]
        estimates.append(Function(u, solution))
    return estimates

In [ ]:
estimates = solve_qp(u, x_grid, predictor, qfs)
# For better visualization in 3d plot
estimates = [estimate.drop_inf() for estimate in estimates]

In [ ]:
for func in estimates[:5]:
    plt.plot(func.x, func.y)

In [ ]:
# Initialize a 2D array to hold f(u, x) values
z_values = np.zeros((len(x_grid), len(func.x)))

# Populate the array with estimates
for i, func in enumerate(estimates):
    z_values[i] = func.y

# Create the 3D surface plot
X, Y = np.meshgrid(func.x, x_grid)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(X, Y, z_values, cmap="viridis")

# Adjust the viewing angle for better visibility
ax.view_init(elev=15, azim=255)

ax.set_xlabel("u")
ax.set_ylabel("Predictor x")
ax.set_zlabel("f(u, x)", rotation=0)
plt.tight_layout()
plt.show()

### Compute Integrated Squared Error

In [ ]:
from frechet_fda.tools.function_tools import quantile_distance

In [ ]:
m_hat = [estimate.drop_inf() for estimate in estimates]

In [ ]:
true_m = qfs[::10]
true_m = [qf.drop_inf() for qf in true_m]

In [ ]:
differences = [hat - true for hat, true in zip(m_hat, true_m, strict=True)]

In [ ]:
distances = [
    quantile_distance(hat, true, already_qf=True)
    for hat, true in zip(m_hat, true_m, strict=True)
]

In [ ]:
distances

In [ ]:
def ise_wasserstein(m_hat, true_m, x_grid):
    """Compute integrated squared error."""
    distances = [
        quantile_distance(hat, true, already_qf=True)
        for hat, true in zip(m_hat, true_m, strict=True)
    ]
    return Function(x_grid, distances).integrate().y[-1]

In [ ]:
ise_wasserstein(m_hat, true_m, x_grid)

## Functional Regression with Transformation method

In [ ]:
from frechet_fda.tools.function_tools import (
    inverse_log_qd_transform,
    log_qd_transform,
)

In [ ]:
mus[:5], sigmas[:5]

In [ ]:
# see whether toying around with function and returning to it changes anything
qfs[199].drop_inf().differentiate().integrate().compare(qfs[199].drop_inf() + 0.1)

In [ ]:
pdfs = [qf.drop_inf().invert().differentiate() for qf in qfs]
pdfs_normalized = [pdf.standardize_support() for pdf in pdfs]
lqdfs_and_start_vals = log_qd_transform(pdfs, different_supports=True)
lqdfs = lqdfs_and_start_vals[:, 0]
start_vals = lqdfs_and_start_vals[:, 1]

In [ ]:
predictor_matrix = np.array((np.ones(len(predictor)), predictor)).transpose()
predictor_matrix.shape, np.linalg.inv(
    predictor_matrix.transpose() @ predictor_matrix,
).shape

In [ ]:
log_betahat = (
    np.linalg.inv(predictor_matrix.transpose() @ predictor_matrix)
    @ predictor_matrix.transpose()
    @ lqdfs
)
log_betahat0 = log_betahat[0]
log_betahat1 = log_betahat[1]
log_betahat0.compare(log_betahat1)

In [ ]:
lqdf_hat = predictor * log_betahat1 + log_betahat0
lqdfs[0].compare(lqdf_hat[0])
lqdfs[199].compare(lqdf_hat[199])
pdf_hat = inverse_log_qd_transform(lqdf_hat, start_vals)
pdfs[0].compare(pdf_hat[0])
pdfs[199].compare(pdf_hat[199])

In [ ]:
mus[0], mus[99], mus[199], sigmas[0], sigmas[99], sigmas[199]

In [ ]:
distribution_of_interest = 180
pdfs[distribution_of_interest].compare(pdf_hat[distribution_of_interest])
qfs[distribution_of_interest].compare(
    pdf_hat[distribution_of_interest].integrate().invert(),
)

In [ ]:
# Number of simulations
m = 5
# Number of samples of densities to generate
sample_sizes = [200]
# Number of sample points to generate from each density for density estimation step
points_number = 100
# Bandwidth choice for density estimation
bandwidth_choice = 0.2
# Fineness of grids to evaluate functions
grid_size = 1000
# Where to truncate normal distribution
trunc = 3
#
threshold_variance_explained = 0.9
# Initialize arrays to store Fréchet and cross sectional means
stored_f_means = np.empty((m, len(sample_sizes)), dtype="object")
stored_cs_means = np.empty((m, len(sample_sizes)), dtype="object")
stored_f_means_denstimation = np.empty((m, len(sample_sizes)), dtype="object")
stored_cs_means_denstimation = np.empty((m, len(sample_sizes)), dtype="object")

In [ ]:
total_iterations = m * len(sample_sizes)  # Total iterations
current_iteration = 0  # Initialize a counter for the current iteration
print("Simulating...", end="\r")
for i in range(m):
    # Try different sample sizes
    for j, n in enumerate(sample_sizes):
        # j index only used for storing Fréchet means below!
        # Set parameters
        seed_num = int(str(i) + str(j))  # unique seed in each simulation run
        mus, sigmas = gen_params_scenario_one(n, seed=seed_num)
        # Sort sigmas just for inspection reasons
        sigmas.sort()

        # Generate pdfs within truncation points
        pdfs = make_truncnorm_pdf(
            -trunc,
            trunc,
            mus,
            sigmas,
            grid_size=grid_size,
            warn_irregular_densities=False,
        )
        # Make Function class objects
        my_pdfs = make_function_objects(pdfs)

        # For numerical correction: shorten the range for smaller sigmas to get rid of
        # numerical artifacts when computing integrals, derivatives and means later
        new_ranges = get_optimal_range(my_pdfs)
        # Generate pdfs again, this time within individual ranges
        pdfs2 = [
            make_truncnorm_pdf(
                new_ranges[i][0],
                new_ranges[i][1],
                mus[i],
                sigmas[i],
                grid_size=grid_size,
                warn_irregular_densities=False,
            )[0]
            for i in range(n)
        ]

        # Generate numerically stable objects
        new_pdfs = make_function_objects(pdfs2)

        # Compute Fréchet mean
        stored_f_means[i, j] = frechet_mean(new_pdfs)
        stored_cs_means[i, j] = mean_func(new_pdfs)

        # Compute Fréchet variance
        total_variance = total_frechet_variance(stored_f_means[i, j], new_pdfs)

        # Transform pdf sample
        log_qdfs = log_qd_transform(new_pdfs)
        (
            mean_log_qdfs,
            eigenvalues_log_qdfs,
            eigenfunctions_log_qdfs,
            fpc_scores_log_qdfs,
        ) = gen_qdtransformation_pcs(log_qdfs, k=2)

        # Try function that finds optimal trunc representation. Threshold: explain
        # at least threshold_variance_explained
        optimal_k, fraction_explained, truncated_representations = k_optimal(
            threshold_variance_explained,
            total_variance,
            new_pdfs,
            mean_log_qdfs,
            eigenfunctions_log_qdfs,
            fpc_scores_log_qdfs,
        )

        ### And now again with additional density estimation step!
        which_kernel = "std_normal"
        sample_points = gen_truncnorm_pdf_points(
            -trunc,
            trunc,
            mus,
            sigmas,
            points_number,
        )
        pdf_hats = make_estimated_truncnorm_pdf(
            sample_points=sample_points,
            a=-trunc * np.ones(n),
            b=trunc * np.ones(n),
            kern=which_kernel,
            grid_size=grid_size,
            bandwidth=bandwidth_choice,
        )

        new_ranges = get_optimal_range(pdf_hats)
        # Generate numerically stable objects, within individual ranges
        new_pdf_hats = make_estimated_truncnorm_pdf(
            sample_points=sample_points,
            a=new_ranges[:, 0],
            b=new_ranges[:, 1],
            kern=which_kernel,
            grid_size=grid_size,
            bandwidth=bandwidth_choice,
        )

        # Compute Fréchet mean
        stored_f_means_denstimation[i, j] = frechet_mean(new_pdf_hats)
        stored_cs_means_denstimation[i, j] = mean_func(new_pdf_hats)

        # Compute Fréchet variance
        total_variance = total_frechet_variance(
            stored_f_means_denstimation[i, j],
            new_pdf_hats,
        )

        # Transform pdf sample
        log_qdfs = log_qd_transform(new_pdf_hats)

        (
            mean_log_qdfs,
            eigenvalues_log_qdfs,
            eigenfunctions_log_qdfs,
            fpc_scores_log_qdfs,
        ) = gen_qdtransformation_pcs(log_qdfs, k=2)

        # Try function that finds optimal trunc representation. Threshold: explain
        # at least threshold_variance_explained
        optimal_k, fraction_explained, truncated_representations = k_optimal(
            threshold_variance_explained,
            total_variance,
            new_pdf_hats,
            mean_log_qdfs,
            eigenfunctions_log_qdfs,
            fpc_scores_log_qdfs,
        )

        # Print progress of simulation
        current_iteration += 1  # Increment the current iteration
        perc = int(100 * current_iteration / total_iterations)
        print(f"Simulating...{perc}%", end="\r")